In [1]:
import numpy as np
import scipy.stats as ss
from scipy.stats import norm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline

from scipy import sparse
from scipy.sparse.linalg import splu
from scipy.sparse.linalg import spsolve

In [2]:
def implicit_fin_diff(S,K,T,sigma,r,N,Nj,CallPut):
    '''
    Implicit finite difference method for pricing European call and put options
        
    Args:
        S - intial price of underlying asset
        K - strike price
        T - time to maturity
        sigma - volatility
        r - risk-free rate
        
        N - number of time intervals (horizontal)
        Nj - number of partition points from 0 to the upper/lower boundary (vertical)
        CallPut - 'Call' or 'Put'
        
    Returns the option price estimated by the finite difference grid    
    
    '''
    dt = T/N;    #dx = sigma*np.sqrt(3*dt)
    Smax= 100
    dx = Smax/Nj
    nu = r - 0.5*sigma**2
    pu = -0.5*dt*((sigma/dx)**2 + nu/dx);   pm = 1.0 + dt*(sigma/dx)**2 + r*dt;    pd = -0.5*dt*((sigma/dx)**2 - nu/dx)
    grid = np.zeros(2*Nj+1)
    
    # Asset prices at maturity:
    St = [S*np.exp(-Nj*dx)]
    for j in range(1, 2*Nj+1):
        St.append(St[j-1]*np.exp(dx))
    
    # Option value at maturity:
    for j in range(2*Nj+1):
        if CallPut == 'Call':
            grid[j] = max(0, St[j] - K)
        elif CallPut == 'Put':
            grid[j] = max(0, K - St[j])
        # Boundary Conditions:
    if CallPut == 'Call':
        lambdaU = St[2*Nj] - St[2*Nj-1];    lambdaL = 0.0;
    elif CallPut == 'Put':
        lambdaU = 0.0;  lambdaL = -1.0*(St[1] - St[0])
    
    # Backwards computing through grid
    def tridiagonal(C,pU,pM,pD,lambda_L,lambda_U,nj):
        '''
        Helper function for solving the tridiagonal matrix system specified by the 
        implicit finite difference method
        '''
        C1 = np.zeros(2*nj+1)     
        pmp = [pM+pD]
        pp = [C[1]+pD*lambda_L]
        for j in range(2,2*nj):
            pmp.append(pM - pU*pD/pmp[j-2])
            pp.append(C[j] - pp[j-2]*pD/pmp[j-2])
        C1[2*nj] = (pp[len(pp)-1] + pmp[len(pmp)-1]*lambda_U)/(pU + pmp[len(pmp)-1])
        C1[2*nj-1] = C1[2*nj] - lambda_U
        for j in range(2*nj-2, -1, -1):
            C1[j] = (pp[j-1] - pU*C1[j+1])/pmp[j-1]
        C1[0] = C1[1] - lambda_L
        return C1
    
    for i in range(N):  
        grid = tridiagonal(grid,pu,pm,pd,lambdaL,lambdaU,Nj)
    
    return grid[Nj]        

In [3]:
EuropeanPut = implicit_fin_diff(40,40,1,0.3,0.1,400,400,'Put')
EuropeanPut

2.4483184375057845

In [4]:
N = norm.cdf
def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)
PutOprice = BS_PUT(40, 40, 1, 0.1, 0.3)
PutOprice

2.8871501543930442

In [5]:
EuropeanPut = implicit_fin_diff(40,40,1,0.3,0.1,15000,30000,'Put')
EuropeanPut

2.8870516949438585

In [6]:
S0 = 40.0  # spot stock price
K = 40.0  # strike
T = 1.0  # maturity
r = 0.1  # risk free rate
sig = 0.3  # diffusion coefficient or volatility
X0 = np.log(S0)  # logprice
B = 30  # Barrier 1


In [7]:
Nspace = 400  # M space steps
Ntime = 400  # N time steps
S_max = 100  # The max of S corresponds to the maximum boundary condition
S_min = 30 # The min of S corresponds to the Barrier which corresponds to minimum boundary condition

x_max = np.log(S_max)  # A2
x_min = np.log(S_min)  # A1

x, dx = np.linspace(x_min, x_max, Nspace, retstep=True)  # space discretization
T_array, dt = np.linspace(0, T, Ntime, retstep=True)  # time discretization
Payoff = np.maximum(K - np.exp(x), 0)  # Put Payoff

V = np.zeros((Nspace, Ntime))  # grid initialization
offset = np.zeros(Nspace - 2)  # vector to be used for the boundary terms

V[:, -1] = Payoff  # terminal conditions
V[-1, :] = 0  # boundary condition
V[0, :] = 0  # boundary condition

# construction of the tri-diagonal matrix D
sig2 = sig * sig
dxx = dx * dx
a = (dt / 2) * ((r - 0.5 * sig2) / dx - sig2 / dxx)
b = 1 + dt * (sig2 / dxx + r)
c = -(dt / 2) * ((r - 0.5 * sig2) / dx + sig2 / dxx)
D = sparse.diags([a, b, c], [-1, 0, 1], shape=(Nspace - 2, Nspace - 2)).tocsc()
DD = splu(D)

# Backward iteration
for i in range(Ntime - 2, -1, -1):
    offset[0] = a * V[0, i]
    offset[-1] = c * V[-1, i]
    V[1:-1, i] = DD.solve(V[1:-1, i + 1] - offset)

# finds the option at S0
oPrice = np.interp(X0, x, V[:, 0])
print("The price of the Down and Out Put option by PDE is: ", oPrice)

The price of the Down and Out Put option by PDE is:  0.5888346955319073
